<a href="https://colab.research.google.com/github/olusolaag/AI-ML-Projects-2021/blob/main/creditcardfraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [3]:
%ls


kaggle.json  sample_data/


In [5]:
#  Install the Kaggle API
!pip install -q kaggle

In [6]:
# Move the kaggle.json file into ~/.kaggle, which is where the API client expects your token to be located:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
#Execute the following command to verify whether the kaggle.json is stored in the appropriate location: ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [8]:
!chmod 600 /root/.kaggle/kaggle.json #run this command to ensure your Kaggle API token is secure on colab

In [9]:
# create folder to download dataset
!mkdir creditfrauddataset

In [10]:
#If you get a forbidden link, you have most likely not joined the competition.
!kaggle datasets download -d mlg-ulb/creditcardfraud -p creditfrauddataset


 91% 60.0M/66.0M [00:01<00:00, 49.6MB/s]
100% 66.0M/66.0M [00:01<00:00, 45.5MB/s]


In [11]:
!unzip "/content/creditfrauddataset/creditcardfraud.zip"

Archive:  /content/creditfrauddataset/creditcardfraud.zip
  inflating: creditcard.csv          


In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import StandardScaler
# from sklearn.compose import ColumnTransformer

In [13]:
df = pd.read_csv("/content/creditcard.csv")
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [13]:
#get the numbers of categories in Class column
classes = pd.unique(df['Class'])
num_of_classes = len(classes)
print('classes:',classes)
print('number of classes:',num_of_classes)

classes: [0 1]
number of classes: 2


In [14]:
#get the number rows and columns in the dataset
rows, columns = df.shape
print('Before Time column was droppd:')
print('number of rows: ', rows)
print('number of columns: ', columns)

Before Time column was droppd:
number of rows:  284807
number of columns:  31


In [15]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [16]:
# drop time column as no significant relevance
df.drop('Time', axis=1, inplace=True)

In [17]:
#get the number rows and columns in the dataset
rows, columns = df.shape
print('After Time column was droppd:')
print('number of rows: ', rows)
print('number of columns: ', columns)

After Time column was droppd:
number of rows:  284807
number of columns:  30


In [18]:
num_of_cases = rows
non_fraud_cases = df[df.Class==0]
fraud_cases = df[df.Class==1]
num_of_non_fraud = len(non_fraud_cases)
num_of_fraud = len(fraud_cases)
percentage_fraud = round(num_of_fraud/num_of_non_fraud*100,2)
print('num_of_cases:', num_of_cases)
print('num_of_non_fraud:', num_of_non_fraud)
print('num_of_fraud:', num_of_fraud)
print('percentage_fraud:', percentage_fraud)

num_of_cases: 284807
num_of_non_fraud: 284315
num_of_fraud: 492
percentage_fraud: 0.17


In [19]:
#non fraud cases info
non_fraud_cases.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [20]:
#fraud cases info
fraud_cases.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [21]:
creditcard_df = df.copy()

In [22]:
creditcard_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [23]:
y = creditcard_df["Class"].values
updated_cc_df = creditcard_df.drop(columns=["Class"])

updated_cc_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [24]:
# list(updated_cc_df.columns)

In [25]:
# column_transf = ColumnTransformer(transformers=[('scaler',StandardScaler(),['Amount'])],remainder='passthrough')
# X = column_transf.fit_transform(X)
scaler = StandardScaler()
amount = updated_cc_df['Amount'].values
updated_cc_df['Amount'] = scaler.fit_transform(amount.reshape(-1,1))


In [26]:
print('updated_cc_df shape: ',updated_cc_df.shape)

updated_cc_df shape:  (284807, 29)


In [27]:
updated_cc_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403


In [28]:
updated_cc_df.shape

(284807, 29)

In [29]:
def remove_Highly_Correlated(df, bar=0.9):
  # Creates correlation matrix
  corr = df.corr()

  # Set Up Mask To Hide Upper Triangle
  mask = np.triu(np.ones_like(corr, dtype=bool))
  tri_df = corr.mask(mask)

  # Finding features with correlation value more than specified threshold value (bar=0.9)
  highly_cor_col = [col for col in tri_df.columns if any(tri_df[col] > bar )]
  print("length of highly correlated columns",len(highly_cor_col))

  # Drop the highly correlated columns
  reduced_df = df.drop(highly_cor_col, axis = 1)
  print("shape of total data",df.shape,"shape of reduced data",reduced_df.shape)
  return reduced_df

In [30]:
#remove highly correllated features
resAfterCorMatrix = remove_Highly_Correlated(updated_cc_df)

length of highly correlated columns 0
shape of total data (284807, 29) shape of reduced data (284807, 29)


In [31]:
updated_cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 29 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      284807 non-null  float64
 1   V2      284807 non-null  float64
 2   V3      284807 non-null  float64
 3   V4      284807 non-null  float64
 4   V5      284807 non-null  float64
 5   V6      284807 non-null  float64
 6   V7      284807 non-null  float64
 7   V8      284807 non-null  float64
 8   V9      284807 non-null  float64
 9   V10     284807 non-null  float64
 10  V11     284807 non-null  float64
 11  V12     284807 non-null  float64
 12  V13     284807 non-null  float64
 13  V14     284807 non-null  float64
 14  V15     284807 non-null  float64
 15  V16     284807 non-null  float64
 16  V17     284807 non-null  float64
 17  V18     284807 non-null  float64
 18  V19     284807 non-null  float64
 19  V20     284807 non-null  float64
 20  V21     284807 non-null  float64
 21  V22     28

In [32]:
X = updated_cc_df.values

In [33]:
# Train and test split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [34]:
#Decision tree
# Create DecisionTree Classifier object 
clf = DecisionTreeClassifier(max_depth=2)

# Train the model 
clf.fit(X_train, y_train) 

# Making predictions on the testing set 
pred = clf.predict(X_test)

y_score_DT = clf.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_DT = accuracy_score(y_test,pred)
f1_score_DT = f1_score(y_test,pred,average='binary')

In [35]:
#calculate auc for model
precision, recall, thresholds = precision_recall_curve(y_test, y_score_DT)
auc_precision_recall_DT = auc(recall, precision)

In [36]:
# Store the accuracy results for each model in a dataframe for final comparison
results = pd.DataFrame({'classifier':['Decision Tree'], 'accuracy': acc_DT,'f1-score': f1_score_DT, 'auc': auc_precision_recall_DT},index={'1'})
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.99921,0.720497,0.732516


In [37]:
#Logistic regression
lgr = LogisticRegression()

# Train the model 
lgr.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_LGR = lgr.predict(X_test)

y_score_LGR = lgr.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_LGR = accuracy_score(y_test,pred_LGR)
f1_score_LGR = f1_score(y_test,pred_LGR,average='binary')

#calculate auc for model
precision_2, recall_2, thresholds_2 = precision_recall_curve(y_test, y_score_LGR)
auc_precision_recall_LGR = auc(recall_2, precision_2)

In [38]:
# Store the accuracy results for each model in a dataframe for final comparison
results_LGR = pd.DataFrame({'classifier':['LogisticRegression'], 'accuracy': acc_LGR,'f1-score': f1_score_LGR, 'auc': auc_precision_recall_LGR},index={'2'})

results = pd.concat([results, results_LGR])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.99921,0.720497,0.732516
2,LogisticRegression,0.99914,0.662069,0.693701


In [39]:
#KNN - K-Nearest Neighbour
n = 5
knn = KNeighborsClassifier(n_neighbors=n)

# Train the model 
knn.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_KNN = knn.predict(X_test)

y_score_KNN = knn.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_KNN = accuracy_score(y_test,pred_KNN)
f1_score_KNN = f1_score(y_test,pred_KNN,average='binary')

#calculate auc for model
precision_4, recall_4, thresholds_4 = precision_recall_curve(y_test, y_score_KNN)
auc_precision_recall_KNN = auc(recall_4, precision_4)

# Store the accuracy results for each model in a dataframe for final comparison
results_KNN = pd.DataFrame({'classifier':['KNN'], 'accuracy': acc_KNN,'f1-score': f1_score_KNN, 'auc': auc_precision_recall_KNN},index={'3'})

results = pd.concat([results, results_KNN])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500


In [40]:
#RandomForest Classifier
rf = RandomForestClassifier(n_estimators=100)

# Train the model 
rf.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_RF = rf.predict(X_test)

y_score_RF = rf.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_RF = accuracy_score(y_test,pred_RF)
f1_score_RF = f1_score(y_test,pred_RF,average='binary')

#calculate auc for model
precision_5, recall_5, thresholds_5 = precision_recall_curve(y_test, y_score_RF)
auc_precision_recall_RF = auc(recall_5, precision_5)

# Store the accuracy results for each model in a dataframe for final comparison
results_RF = pd.DataFrame({'classifier':['RandomForestClassifier'], 'accuracy': acc_RF,'f1-score': f1_score_RF, 'auc': auc_precision_recall_RF},index={'4'})

results = pd.concat([results, results_RF])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500
4,RandomForestClassifier,0.999544,0.835443,0.800822


In [41]:
#XGBoost classifier
xgb = XGBClassifier(max_depth=4)

# Train the model 
xgb.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_XGB = xgb.predict(X_test)

y_score_XGB = xgb.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_XGB = accuracy_score(y_test,pred_XGB)
f1_score_XGB = f1_score(y_test,pred_XGB,average='binary')

#calculate auc for model
precision_5, recall_5, thresholds_5 = precision_recall_curve(y_test, y_score_RF)
auc_precision_recall_XGB = auc(recall_5, precision_5)

# Store the accuracy results for each model in a dataframe for final comparison
results_XGB = pd.DataFrame({'classifier':['XGBClassifier'], 'accuracy': acc_XGB,'f1-score': f1_score_XGB, 'auc': auc_precision_recall_XGB},index={'5'})

results = pd.concat([results, results_XGB])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500
4,RandomForestClassifier,0.999544,0.835443,0.800822
5,XGBClassifier,0.999526,0.832298,0.800822


In [42]:
#BaggingClassifier
bg = BaggingClassifier(DecisionTreeClassifier(), n_estimators=20)

# Train the model 
bg.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_BG = bg.predict(X_test)

y_score_BG = bg.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_BG = accuracy_score(y_test,pred_BG)
f1_score_BG = f1_score(y_test,pred_BG,average='binary')

#calculate auc for model
precision_6, recall_6, thresholds_6 = precision_recall_curve(y_test, y_score_BG)
auc_precision_recall_BG = auc(recall_6, precision_6)

# Store the accuracy results for each model in a dataframe for final comparison
results_BG = pd.DataFrame({'classifier':['BaggingClassifier'], 'accuracy': acc_BG,'f1-score': f1_score_BG, 'auc': auc_precision_recall_BG},index={'6'})

results = pd.concat([results, results_BG])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500
4,RandomForestClassifier,0.999544,0.835443,0.800822
5,XGBClassifier,0.999526,0.832298,0.800822
6,BaggingClassifier,0.999526,0.828025,0.788570


In [44]:
#adaBoostClassifier
adb = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=10)

# Train the model 
adb.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_ADB = adb.predict(X_test)

y_score_ADB = adb.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_ADB = accuracy_score(y_test,pred_ADB)
f1_score_ADB = f1_score(y_test,pred_ADB,average='binary')

#calculate auc for model
precision_7, recall_7, thresholds_7 = precision_recall_curve(y_test, y_score_ADB)
auc_precision_recall_ADB = auc(recall_7, precision_7)

# Store the accuracy results for each model in a dataframe for final comparison
results_ADB = pd.DataFrame({'classifier':['AdaBoostClassifier'], 'accuracy': acc_ADB,'f1-score': f1_score_ADB, 'auc': auc_precision_recall_ADB},index={'7'})

results = pd.concat([results, results_ADB])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500
4,RandomForestClassifier,0.999544,0.835443,0.800822
5,XGBClassifier,0.999526,0.832298,0.800822
6,BaggingClassifier,0.999526,0.828025,0.788570
7,AdaBoostClassifier,0.999175,0.728324,0.728559
7,AdaBoostClassifier,0.999157,0.720930,0.721247


In [45]:
#VotingClassifier (Ensemble)
vc = VotingClassifier(estimators=[('dt',DecisionTreeClassifier()), ('lgr', LogisticRegression())], voting='soft')

# Train the model 
vc.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_VC = vc.predict(X_test)

y_score_VC = vc.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_VC = accuracy_score(y_test,pred_VC)
f1_score_VC = f1_score(y_test,pred_VC,average='binary')

#calculate auc for model
precision_8, recall_8, thresholds_8 = precision_recall_curve(y_test, y_score_VC)
auc_precision_recall_VC = auc(recall_8, precision_8)

# Store the accuracy results for each model in a dataframe for final comparison
results_VC = pd.DataFrame({'classifier':['Ensemble'], 'accuracy': acc_VC,'f1-score': f1_score_VC, 'auc': auc_precision_recall_VC},index={'8'})

results = pd.concat([results, results_VC])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500
4,RandomForestClassifier,0.999544,0.835443,0.800822
5,XGBClassifier,0.999526,0.832298,0.800822
6,BaggingClassifier,0.999526,0.828025,0.788570
7,AdaBoostClassifier,0.999175,0.728324,0.728559
7,AdaBoostClassifier,0.999157,0.720930,0.721247
8,Ensemble,0.999298,0.764706,0.777361


In [46]:
#VotingClassifier (Ensemble_2)
vc_top2 = VotingClassifier(estimators=[('rf',RandomForestClassifier()), ('xgb', XGBClassifier())], voting='soft')

# Train the model 
vc_top2.fit(X_train, y_train) 

# Making predictions on the testing set 
pred_VC2 = vc_top2.predict(X_test)

y_score_VC2 = vc_top2.predict_proba(X_test)[:,1]

# Comparing actual response values (y_test)
# with predicted response values (y_pred) 
acc_VC2 = accuracy_score(y_test,pred_VC2)
f1_score_VC2 = f1_score(y_test,pred_VC2,average='binary')

#calculate auc for model
precision_9, recall_9, thresholds_9 = precision_recall_curve(y_test, y_score_VC2)
auc_precision_recall_VC2 = auc(recall_9, precision_9)

# Store the accuracy results for each model in a dataframe for final comparison
results_VC2 = pd.DataFrame({'classifier':['Ensemble_2'], 'accuracy': acc_VC2,'f1-score': f1_score_VC2, 'auc': auc_precision_recall_VC2},index={'9'})

results = pd.concat([results, results_VC2])
results = results[['classifier', 'accuracy', 'f1-score', 'auc' ]]
results

,classifier,accuracy,f1-score,auc
1,Decision Tree,0.999210,0.720497,0.732516
2,LogisticRegression,0.999140,0.662069,0.693701
3,KNN,0.999438,0.794872,0.829500
4,RandomForestClassifier,0.999544,0.835443,0.800822
5,XGBClassifier,0.999526,0.832298,0.800822
6,BaggingClassifier,0.999526,0.828025,0.788570
7,AdaBoostClassifier,0.999175,0.728324,0.728559
7,AdaBoostClassifier,0.999157,0.720930,0.721247
8,Ensemble,0.999298,0.764706,0.777361
9,Ensemble_2,0.999579,0.850000,0.793563
